# TP 14: Predicting dynamics with Neural-ODE


**The goal of this pratical is use machine learning models to predict the evolution of dynamical systems driven by physical laws, *e.g.* ordinary Differential Equations (ODE).**

Let us considers a physcial system in Newtonian mechanichs composed of a **damped pendulum**, with length $l$ and mass $m$, and $\theta$ being the angle with respect to the vertical direction:
<img src="./pendulum.png" width="200">

**Let us denote $\dot{\theta_t}:=\frac{d\theta}{dt}$ and $\ddot{\theta}_t:=\frac{d^2\theta}{dt^2}$ as the first and second temporal derivatives of $\theta$.** The dynamics of the pendulum is driven bt the following ODE on $\theta$:


\begin{equation} \ddot{\theta_t} + \omega_0^2~ sin\left(\theta_t\right) + \alpha \dot{\theta}_t = 0 \label{eq1}\tag{1},
\end{equation}

where $\omega_0 = \sqrt{\frac{g}{l}}$ ($g$ is the gravitational constant), and $\alpha = \frac{k}{ml^2}$ is the friction coefficient.

In the general case, the ODE in Eq (\ref{eq1}) does not have a closed-form solution. Let us denote as $\mathbf{Y}_t=(\theta_t, \dot{\theta}_t)$ the 2d state vector of the pendulum. 



 **<u>Question 1:</u> show that $\dot{\mathbf{Y}_t}=f\left({\mathbf{X}_t}\right)$, *i.e* that the evolution of $\mathbf{Y}$ follows a first-order ODE. Give the expression of f.** 

From a given initial condition $\mathbf{Y}_0=(\theta_0, \dot{\theta}_0)$, we can estimate the state vector $\mathbf{Y}_t$ at any time $t$: 

\begin{equation}
\mathbf{Y}_t = \mathbf{Y}_0 + \int_0^t \dot{\mathbf{Y}_t} ~dt = \mathbf{Y}_0 + \int_0^t f\left(\mathbf{Y}_t\right) dt \label{eq2}\tag{2},
\end{equation}

where $f\left( \mathbf{Y}_t \right)$ only depends on the current state $\mathbf{Y}_t$ at time $t$. The integral in Eq (\ref{eq2}) can be approximated with numerical schemes. The Euler method is simplest one (see figure below): starting from $\mathbf{Y}_0$, we have $\mathbf{Y}_{t+1} = \mathbf{Y}_{t} + f\left(\mathbf{Y}_t\right)$ $\forall t>1$. The has been extensive studies for developping improved numerical solvers in the last centuries, e.g. different orders of Runge-Kutta solvers.
<img src="./Euler.png" width="200">

## Part I. Generating damped pendulum simulations
First, lets do some import

In [ ]:
import math, shelve
import os

from collections import OrderedDict

import torch
import numpy as np
from scipy.integrate import solve_ivp

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim

from torchdiffeq import odeint_adjoint, odeint

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]


### I.a) DampledPendulum

**We will write a DampledPendulum Dataset, which simulates different pendulum trajectories from differents initial conditions. Fill the code in the code in the following DampledPendulum class. We use the following setting:** $\omega_0^2= \frac{\Pi}{6}$, $\alpha= 0.2$, time hoziron : 10, with $dt=0.5$. 

You have to fill the __init__, __len__ and  __getitem__ functions. For __getitem__, the goal is to simulate a given trajectory from an initial condition: 
- The function _get_initial_condition is provided
- To perform the simulation in __getitem__, you need to: 
    - Call the _get_initial_condition
    - Call the solver: we will use the solve_ivp method from from scipy.integrate, using the 'DOP853' method (Explicit Runge-Kutta method of order 8). 
- Since the simulation is computationnaly demanding, it can be a good idea to store the states in the class

In [ ]:
class DampledPendulum(Dataset):
    def __init__(self, num_seq, time_horizon, dt):
    
        super().__init__()
        
        self.omega0_square= 0.0 # FILL WITH YOUR CODE
        self.alpha= 0.0 # FILL WITH YOUR CODE
        
        self.len = 0.0 # NUMBER OF SEQUENCES IN DATASET - FILL WITH YOUR CODE
        self.time_horizon 0.0 # FILL WITH YOUR CODE
        self.dt 0.0 # FILL WITH YOUR CODE
        self.data ={} 0.0 # TO STORE THE STATES


    def _get_initial_condition(self, seed):
        y0 = np.random.randn(2) * 2.0 - 1
        r = np.random.rand() + 1.3
        y0 = y0 / np.sqrt((y0 ** 2).sum()) * r
        
        return y0
        
    def __getitem__(self, index):
        t_eval = torch.from_numpy(np.arange(0, self.time_horizon, self.dt))
        
        if self.data.get(str(index)) is None:
            # GET INITIAL CONDITIONS 
            y0 = self._get_initial_condition(index)
            # PERFORM SIMULATION (i.e. NUMERICAL INTEGRATION) - FILL WITH YOUR CODE  
        else:
            # LOAD ALREADY COMPUTED STATES - FILL WITH YOUR CODE
        
        return {'states': states, 't': t_eval.float()}

    def __len__(self):
        # FILL WITH YOUR CODE
        return 0

### I.b) Train/test data generation
**We can now define train and test dataloader** (use 25 train/test sequences with a batch size of 25).
**Plot the resulting trajectories ($\theta$ and optionally $\dot{\theta}$).**

## 2) Predicting trajectories with Neural-ODE

**The goal is to use the Neural-ODE method [1] to predict the future trajectory from an initial condition.** As mentionned before, the idea is to define a parametric model to predict the state's derivative from the current state value.

**Let's fill the DerivativeEstimator class to predict the the state's derivative.** We will use a simple MLP (2 hiddenn layers + ReLU) for prediction since the state is a 2D vector. 


In [ ]:
class DerivativeEstimator(nn.Module):
    def __init__(self, n_state , n_hidden):
        super().__init__()
        # FILL WITH YOUR CODE

    def forward(self, t, state):
        # FILL WITH YOUR CODE
        pass

**The forecasterwill perform the prediction from a initial state $y_0$**. To perform the numerical integration, we use the 'odeint' method from torchdiffeq. We will use the generic 'rk4' solver to perform numerical integration. **Fill the following  Forecaster class with:** 
- A constructor creating a reference to an DerivativeEstimator instance 
- the forward method calls the odeint method to perform integration from an initial $y_0$ state. **N.B.**: the output dimensions after calling odeint will be T x batch_size x n_c, swap them to fit the requested Pytorch standard (batch_size x n_c X T)

In [ ]:
class Forecaster(nn.Module):
    def __init__(self, n_state , n_hidden, method='rk4'):
        super().__init__()
        
    def forward(self, y0, t):
        # CALL TO ODEINT + DIM SWAP
        pass

### Write the training loop!
For each batch: 
- Get the first state of each training trajectory
- Perform prediction of the forecaster for each time step of the horizon
- We will use a simple MSE loss between the ground truth and predicted trajectories.
- Use an Adam optimizer (default paramters)
- Plot the train / test trajectories

In [ ]:
n_state = 2
n_hidden = 200
n_epochs = 1001


for e in range(n_epochs):
    for iteration,batch in enumerate(train_loader): 
        # FILL WITH YOUR CODE


# Bonus
Experiment Neural ODE for **replacing residual networks with ODEs for supervised learning**: see section 3 in [this paper](https://proceedings.neurips.cc/paper/2018/file/69386f6bb1dfed68692a24c8686939b9-Paper.pdf). 

[1] **Neural Ordinary Differential Equations.**
Ricky T. Q. Chen, Yulia Rubanova, Jesse Bettencourt, David K. Duvenaud.
NeurIPS 2018.